In [1]:
print("Importing dependencies..")
%load_ext autoreload
%autoreload 2
import scipy
import numpy as np
from scipy.linalg import lstsq
from scipy.linalg import norm 
import pandas as pd
import os
# from util import RegressionGame
# from util_sparse import getShapleyProjection
import os
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from tqdm import tqdm
# import shap
# import llm_helper
from scipy import special
if not os.path.exists('data'):
    os.makedirs('data')
if not os.path.exists('__dcache__'):
    os.makedirs('__dcache__')

# N = 6000
# print("Explanation count: %s" % N)
# k = 100
# print("SHAP sample count: %s" % k)
seq_len = 50
print("Seq Length: %s" % seq_len)

# Setup
np.random.seed(1)
# model = llm_helper.get_model()
cuda = True


Importing dependencies..
Seq Length: 50


# Experiment 3

In [ ]:
import experiment_runner
bert_experiment = experiment_runner.ExperimentRunner(cuda=cuda, seq_len=seq_len, model_name='bert')
bert_experiment.prepare_data()
bert_experiment.prepare_model()

In [ ]:
bert_experiment.run_mwe_interactions()
bert_experiment.run_avg_interactions()

  3%|████▌                                                                                                                                                                | 1612/58830 [04:22<3:27:56,  4.59it/s]

In [2]:

from transformers import BertForMaskedLM, BertTokenizerFast
if cuda:
    model = BertForMaskedLM.from_pretrained('bert-base-uncased').cuda()
else:
    model = BertForMaskedLM.from_pretrained("bert-base-uncased")

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', use_fast=True)
test = pd.concat([pd.read_pickle(f'../mwe_tagger/bert_bert.pkl_{i}').drop(columns= ['syntactic_distance_idx_mapped', 'syntactic_distance_idx', 'lemmas', 'd', 'toks', 'tags']) for i in range(4)])
test['length'] = test['sentence'].str.split().str.len()
test = test[~((test['weak_mwe'].str.len()==0) & (test['strong_mwe'].str.len()==0))]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
test = test.drop(columns = ['syntactic_distance_idx', 'lemmas', 'd', 'toks', 'tags'], errors='ignore')

In [4]:
def get_prediction_softmax(model, X):
    logits = model(X).logits
    abc =  logits[0, :, :].softmax(dim=-1)
    if not cuda:
        return abc.detach().numpy()
    else:
        return abc
    
def interaction_value_di(model, X, tokens):
    token1, token2 = tokens
    # token_next = max(token1, token2) + 1
    AB = get_prediction_softmax(model, X)
    X_t1 = X.clone()
    X_t1[0, token1] = tokenizer.pad_token_id
    A = get_prediction_softmax(model, X_t1)
    
    X_t2 = X.clone()
    X_t2[0,token1] = tokenizer.pad_token_id
    B = get_prediction_softmax(model, X_t2)

    X_t12 = X.clone()
    X_t12[0,token2] = tokenizer.pad_token_id
    X_t12[0,token1] = tokenizer.pad_token_id
    phi = get_prediction_softmax(model, X_t12)

    # print(AB, A, B, phi)
    val = AB - A - B + phi
    
    if cuda:
        val = torch.divide(torch.linalg.norm(val, dim=1), torch.linalg.norm(AB, dim=1)).cpu()
    else:
        val = np.linalg.norm(val, axis=1)

    if cuda:
        return val.detach()
    else:
        return val



def mwe_distance_interaction(encoded_row, row, col, row_number):

    iv_mwe = []
    
    mwes = row[col]
    encoded_row = encoded_row.reshape(1, -1)
    for mwe in mwes:

        for i in range(len(mwe)):
            for j in range(len(mwe)):
                if i > j:
                    if len([x for x in mwe if x >= seq_len]) > 0:
                        continue
                    iv = interaction_value_di(model, encoded_row, [mwe[i]-1, mwe[j]-1])
                    iv_mwe.append([iv, abs((mwe[i]-1-(mwe[j]-1))),mwe, row_number, i, j])
                    
    return iv_mwe

weak_mwe_distance = []
strong_mwe_distance = []

for row_number, row in tqdm(test.iterrows(), total=test.shape[0]):
    abc =  tokenizer(row['sentence'], padding=False,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids[0]
    if cuda:
        abc = abc.cuda()

    if len(row['weak_mwe']) != 0:
        weak_mwe_distance.append(mwe_distance_interaction(abc, row, 'weak_mwe', row_number))
    
    if len(row['strong_mwe']) != 0:
        strong_mwe_distance.append(mwe_distance_interaction(abc, row, 'strong_mwe', row_number))
    del abc

 50%|█████████████████████████████████████████████████████████████████████████████████                                                                                 | 29432/58830 [1:17:04<1:16:59,  6.36it/s]


KeyboardInterrupt: 

In [5]:
import pickle
# pickle.dump(strong_mwe_distance, open('test_bert_exp3_strong.pkl','wb'))
# pickle.dump(weak_mwe_distance, open('test_bert_exp3_weak.pkl','wb'))
weak_mwe_distance = [y for x in weak_mwe_distance for y in x]
strong_mwe_distance = [y for x in strong_mwe_distance for y in x]

pd.DataFrame(weak_mwe_distance, columns = ['tensor', 'posdis', 'ignore', 'row_number', 'first_token', 'second_token']).to_pickle('weak_mwe_distance_3d_mlm_run2.pkl')
pd.DataFrame(strong_mwe_distance, columns = ['tensor', 'posdis', 'ignore', 'row_number', 'first_token', 'second_token']).to_pickle('strong_mwe_distance_3d_mlm_run2.pkl')

In [ ]:
import os
import datetime
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("Fetching Average Distance...")
cuda = True


def calculate_interaction(encoded_row, row, row_number):
    interactions = []
    encoded_len = len(encoded_row)
    for j in range( min(seq_len, encoded_len)):
        for k in range(j+1, min(seq_len, encoded_len, j+10)):
            if j+k >= encoded_len:
                continue
            if encoded_row[j] == tokenizer.unk_token_id or encoded_row[k] == tokenizer.unk_token_id:
                continue

#             for pred_ind in range(len(encoded_row)):
#                 if pred_ind != j and pred_ind!= k and (abs(pred_ind - max(k, j))<=10 or abs(pred_ind - min(k, j))<=10):
            og = encoded_row.clone()
#             og[pred_ind] = tokenizer.mask_token_id
            og = og.reshape(1, -1)
            iv = interaction_value_di(model, og, [j, k])
            interactions.append([iv, abs((j-k)), row_number, j, k])
            
            # iv = interaction_value_di(model, encoded_row, [j, k])
            # interactions.append([iv, abs(k -j),row_number, [j,k]])
    return interactions

average_distance = []
start_time =  datetime.datetime.now()
# min_row = average_distance[-1][2]
# print(min_row)

for row_number, row in tqdm(test.iterrows(), total=test.shape[0]):
    abc =  tokenizer(row['sentence'], padding=False,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids[0]
    if cuda:
        abc = abc.cuda()

    
    average_distance.extend(calculate_interaction(abc, row, row_number))
    if row_number % 1000 == 0:
        print(len(average_distance))
        pickle.dump(average_distance, open('average_distance_bert_exp3.pkl','wb'))

Fetching Average Distance...


  1%|▉                                                                                                     | 526/58830 [41:51<81:44:43,  5.05s/it]

65236


  3%|██▊                                                                                                | 1640/58830 [2:25:12<42:10:49,  2.66s/it]

218453


...

 18%|█████████████████▋                                                                               | 10738/58830 [16:30:38<69:01:25,  5.17s/it]

1438296


 21%|████████████████████▋                                                                            | 12527/58830 [19:09:40<74:56:29,  5.83s/it]

1669970


 23%|██████████████████████                                                                          | 13482/58830 [20:41:36<102:10:14,  8.11s/it]

In [ ]:
asdasdasd

# Experiment 2

In [4]:
import pandas as pd
cuda = True

from transformers import GPT2LMHeadModel, GPT2TokenizerFast
if cuda:
    model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
else:
    model = GPT2LMHeadModel.from_pretrained('gpt2')

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
# test = pd.read_pickle('../mwe_tagger/fsample.pkl')

test = pd.concat([pd.read_pickle(f'../mwe_tagger/gpt_gpt.pkl_{i}').drop(columns= ['syntactic_distance_idx_mapped', 'syntactic_distance_idx', 'lemmas', 'd', 'toks', 'tags']) for i in range(4)])
# test = pd.read_pickle('../mwe_tagger/wikitext_train_gpt.pkl')
test['length'] = test['sentence'].str.split().str.len()
test = test[~((test['weak_mwe'].str.len()==0) & (test['strong_mwe'].str.len()==0))]

In [5]:
# from joblib import Parallel, delayed
# num_cores = 10

def get_prediction_softmax(model, X, token_next):
    if cuda:
        return torch.softmax(model(X).logits[token_next:], dim=1)
    else:
        return scipy.special.softmax(model(X).logits[token_next:].detach().numpy(), axis=1)

def interaction_value_di(model, X, tokens):
    token1, token2 = tokens
    
    token_next = max(token1, token2) + 1
    AB = get_prediction_softmax(model, X, token_next)
    
    X_t1 = X.clone()
    X_t1[token1] = tokenizer.unk_token_id
    A = get_prediction_softmax(model, X_t1, token_next)
    
    X_t2 = X.clone()
    X_t2[token2] = tokenizer.unk_token_id
    B = get_prediction_softmax(model, X_t2, token_next)

    X_t12 = X.clone()
    X_t12[token2] = tokenizer.unk_token_id
    X_t12[token1] = tokenizer.unk_token_id
    phi = get_prediction_softmax(model, X_t12, token_next)
    
    val = AB - A - B + phi
    if cuda:
        val = torch.linalg.norm(val, dim=1).cpu()
    else:
        val = np.linalg.norm(val, axis=1)
    
    del X_t1, X_t2, X_t12,  AB, A, B, phi
    if cuda:
        return val.detach(), token_next
    else:
        return val, token_next

def mwe_distance_interaction(encoded_row, row, col, row_number):
    iv_mwe = []
    # col = 'weak_mwe' | 'strong_mwe'
    
    mwes = row[col]
    for mwe in mwes:

        for i in range(len(mwe)):
            for j in range(len(mwe)):
                if i > j:
                    if len([x for x in mwe if x >= seq_len]) > 0:
                        continue
                    
                    iv = interaction_value_di(model, encoded_row, [mwe[i]-1, mwe[j]-1])
                    iv_mwe.append([iv, abs((mwe[i]-1-(mwe[j]-1))),mwe, row_number, i, j])
    return iv_mwe

weak_mwe_distance = []
strong_mwe_distance = []


for row_number, row in tqdm(test.iterrows(), total=test.shape[0]):
    # Tokenize each sentence with no padding and truncate if seq_len < tokenized sentence length
    abc =  tokenizer(row['sentence'], padding=False,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids[0]
    if cuda:
        abc = abc.cuda()

    if len(row['weak_mwe']) != 0:
        weak_mwe_distance.append(mwe_distance_interaction(abc, row, 'weak_mwe', row_number))
    
    if len(row['strong_mwe']) != 0:
        strong_mwe_distance.append(mwe_distance_interaction(abc, row, 'strong_mwe', row_number))
    
    del abc

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 58830/58830 [2:32:17<00:00,  6.44it/s]


In [6]:
import pickle
pickle.dump(strong_mwe_distance, open('test1.pkl','wb'))
pickle.dump(weak_mwe_distance, open('test2.pkl','wb'))
weak_mwe_distance = [y for x in weak_mwe_distance for y in x]
strong_mwe_distance = [y for x in strong_mwe_distance for y in x]
pd.DataFrame(weak_mwe_distance, columns = ['I', 'posdis', 'ignore', 'row_number', 'first_token', 'second_token']).to_pickle('weak_mwe_distance_3d_full_run2.pkl')
pd.DataFrame(strong_mwe_distance, columns = ['I', 'posdis', 'ignore', 'row_number', 'first_token', 'second_token']).to_pickle('strong_mwe_distance_3d_full_run2.pkl')

In [ ]:
pd.DataFrame(strong_mwe_distance, columns = ['I', 'posdis', 'ignore', 'row_number', 'first_token', 'second_token'])

In [ ]:
import os
import datetime
# os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("Fetching Average Distance...")
cuda = True


def calculate_interaction(encoded_row, row, row_number):
    interactions = []
    encoded_len = len(encoded_row)
    for j in range( min(seq_len, encoded_len)):
        for k in range(j+1, min(seq_len, encoded_len, j+10)):
            if j+k >= encoded_len:
                continue
            if encoded_row[j] == tokenizer.unk_token_id or encoded_row[k] == tokenizer.unk_token_id:
                continue
            
            iv = interaction_value_di(model, encoded_row, [j, k])
            interactions.append([iv, abs(k -j),row_number, [j,k]])
    return interactions

average_distance = []
start_time =  datetime.datetime.now()
# min_row = average_distance[-1][2]
# print(min_row)

for row_number, row in tqdm(test.iterrows(), total=test.shape[0]):
    # if i%9 == 0 or i<=100:
    # print(i, len(test), f'{i*100/len(test)}%', (datetime.datetime.now() - start_time).seconds)
    # start_time = datetime.datetime.now()
    abc =  tokenizer(row['sentence'], padding=False,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids[0]
    if cuda:
        abc = abc.cuda()

    
    average_distance.extend(calculate_interaction(abc, row, row_number))
    if row_number % 1000 == 0:
        print(len(average_distance))

        if len(average_distance) >= 457902:
            break

# from joblib import Parallel, delayed
# 
# num_cores = 9
# for x in range(0, len(test), 99):
#     print(x, len(test), f'{100*x/len(test)}%')
#     fabc =  lambda x : tokenizer(x, padding=False,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids[0]
    
#     average_distance.extend(Parallel(n_jobs=num_cores)(
#         delayed(calculate_interaction)(fabc(row['sentence']), row, i) for i, row in tqdm(test[x:x+99].iterrows())))

In [ ]:
pd.DataFrame(average_distance, columns = ['I', 'posdis', 'row_number', 'word_pair']).to_pickle('average_distance_3d_full.pkl')

### DATA

In [ ]:
test = pd.read_pickle('../mwe_tagger/fsample.pkl')
a = np.unique([y[-1]-y[0] for x in test['_'].to_list() for y in x], return_counts=True)
print({x:y for x,y in zip(a[0], a[1][::-1].cumsum()[::-1])})
print(np.unique([z for x in test['_'].to_list() for y in x for z in y], return_counts=True))

# EXPERIMENT 1

In [ ]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2', use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
test = pd.read_pickle('../mwe_tagger/fsample.pkl')
test['length'] = test['sentence'].str.split().str.len()
test = test[test['length'] > seq_len].copy().reset_index(drop=True)
X = tokenizer(test['sentence'].to_list(), padding=True,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids
print(X.shape)

In [ ]:
# from joblib import Parallel, delayed
# num_cores = 10

def get_prediction_softmax(model, X):
    return scipy.special.softmax(model(X).logits[-1].detach().numpy())

def interaction_value_di(model, X, tokens):
    token1, token2 = tokens
    AB = get_prediction_softmax(model, X)
    
    X_t1 = X.clone()
    X_t1[token1] = tokenizer.unk_token_id
    A = get_prediction_softmax(model, X_t1)
    
    X_t2 = X.clone()
    X_t2[token2] = tokenizer.unk_token_id
    B = get_prediction_softmax(model, X_t2)

    
    X_t12 = X.clone()
    X_t12[token2] = tokenizer.unk_token_id
    X_t12[token1] = tokenizer.unk_token_id
    phi = get_prediction_softmax(model, X_t12)
    
    val = AB - A - B + phi
    val = np.linalg.norm(val)
    return val

def mwe_distance_interaction(encoded_row, row, col):
    iv_mwe = []
    # col = 'weak_mwe' | 'strong_mwe'
    mwes = row[col]
    for mwe in mwes:

        for i in range(len(mwe)):
            for j in range(len(mwe)):
                if i > j:
                    if len([x for x in mwe if x >= seq_len]) > 0:
                        continue
                    iv = interaction_value_di(model, encoded_row, [mwe[i]-1, mwe[j]-1])
                    iv_mwe.append([iv, abs((mwe[i]-1-(mwe[j]-1))),mwe])
    return iv_mwe
weak_mwe_distance = []
strong_mwe_distance = []

# weak_mwe_distance = Parallel(n_jobs=num_cores)(
#     delayed(mwe_distance_interaction)(X[i], row, 'weak_mwe') for i, row in test.iterrows())

# strong_mwe_distance = Parallel(n_jobs=num_cores)(
#     delayed(mwe_distance_interaction)(X[i], row, 'strong_mwe') for i, row in test.iterrows())


for i, row in test.iterrows():
    if i%100==0:
        print(i, len(test), f'{i*100/len(test)}%')
    weak_mwe_distance.extend(mwe_distance_interaction(X[i], row, 'weak_mwe'))
    strong_mwe_distance.extend(mwe_distance_interaction(X[i], row, 'strong_mwe'))

pd.DataFrame(weak_mwe_distance, columns = ['I', 'posdis', 'ignore']).to_pickle('weak_mwe_distance1.pkl')
pd.DataFrame(strong_mwe_distance, columns = ['I', 'posdis', 'ignore']).to_pickle('strong_mwe_distance1.pkl')

In [ ]:
import os
import datetime
os.environ["TOKENIZERS_PARALLELISM"] = "false"
print("Fetching Average Distance...")

from joblib import Parallel, delayed

def calculate_interaction(encoded_row, row):
    interactions = []
    for j in range( seq_len):
        for k in range(j+1, seq_len):

            if j+k >= seq_len:
                continue
            if encoded_row[j] == tokenizer.unk_token_id or encoded_row[k] == tokenizer.unk_token_id:
                continue
            
            iv = interaction_value_di(model, encoded_row, [j, k])
            interactions.append([iv, abs(k -j),[j,k]])
    return interactions

average_distance = []
# start_time =  datetime.datetime.now()
# for i, row in test.iterrows():
#     # if i%9 == 0 or i<=100:
#     print(i, len(test), f'{i*100/len(test)}%', (datetime.datetime.now() - start_time).seconds)
#     start_time = datetime.datetime.now()
#     average_distance.extend(calculate_interaction(X[i], row))

num_cores = 9
for x in range(0, len(test), 99):
    print(x, len(test), f'{100*x/len(test)}%')
    average_distance.extend(Parallel(n_jobs=num_cores)(
        delayed(calculate_interaction)(X[i], row) for i, row in tqdm(test[x:x+99].iterrows())))


In [ ]:
avg_dist = [item for sublist in average_distance for item in sublist]
pd.DataFrame(avg_dist, columns = ['I', 'posdis', 'ignore']).to_pickle('average_distance1.pkl')

In [ ]:

# avg_dist = [item for sublist in average_distance for item in sublist]

def plot_mean(avg_path='average_distance.pkl', weak_path='weak_mwe_distance.pkl', strong_path='strong_mwe_distance.pkl'):
    avg_dist = pd.read_pickle(avg_path)
    avg_df = pd.DataFrame(avg_dist, columns = ['I', 'posdis', 'ignore']).groupby('posdis')['I'].agg( ['mean', 'count', 'std']).rename(columns = {'mean':'avg_mean', 'count':'avg_count', 'std':'avg_std'})
    weak_mwe_distance = pd.read_pickle(weak_path)
    weak_mwe_df = pd.DataFrame(weak_mwe_distance, columns = ['I', 'posdis', 'ignore']).groupby('posdis')['I'].agg( ['mean', 'count', 'std']).rename(columns = {'mean':'weak_mean', 'count':'weak_count', 'std':'weak_std'})
    weak_mwe_df = weak_mwe_df.drop(0)
    strong_mwe_distance = pd.read_pickle(strong_path)
    strong_mwe_df = pd.DataFrame(strong_mwe_distance, columns = ['I', 'posdis', 'ignore']).groupby('posdis')['I'].agg( ['mean', 'count', 'std']).rename(columns = {'mean':'strong_mean', 'count':'strong_count', 'std':'strong_std'})
    abc = pd.concat([avg_df, weak_mwe_df, strong_mwe_df], axis=1)
    display(abc)
    abc = abc[abc['weak_count'] >=50]
    abc[['avg_mean', 'weak_mean']].plot()
    plt.show()
plot_mean()

In [ ]:
def boxplot_posdis(avg_path='average_distance.pkl', weak_path='weak_mwe_distance.pkl', strong_path='strong_mwe_distance.pkl'):
    avg_dist = pd.read_pickle(avg_path)
    weak_mwe_distance = pd.read_pickle(weak_path)
    strong_mwe_distance = pd.read_pickle(strong_path)
    
    avg_df = pd.DataFrame(avg_dist, columns = ['I', 'posdis', 'ignore']).drop(columns = ['ignore']).assign(Location='avg')
    weak_df = pd.DataFrame(weak_mwe_distance, columns = ['I', 'posdis', 'ignore']).drop(columns = ['ignore']).assign(Location='weak')
    weak_df = weak_df[weak_df['posdis']!=0].copy()
    strong_df = pd.DataFrame(strong_mwe_distance, columns = ['I', 'posdis', 'ignore']).drop(columns = ['ignore']).assign(Location='strong')
    cdf = pd.concat([avg_df, weak_df])#, strong_df])    
    cdf = cdf[cdf['posdis']<=5].copy()
    import seaborn as sns
    import matplotlib.pyplot as plt
    ax = sns.boxplot(x="posdis", y='I',hue="Location", data=cdf)     #hue="Letter",
    plt.show()

boxplot_posdis()

In [ ]:
# avg_df.boxplot(column = ['I'], by = 'posdis')

In [ ]:
# weak_df

In [ ]:
model(X[0:k])

In [ ]:


[x for x in testlist_text if 'Although he wrote' in x]


In [ ]:
pred_mode = 1 # [norm of softmax ]


# TODO: Fix the prediction fn. Might have to incorporate
#  target variable to get the logit for the target variable instead of max
predict_fn = llm_helper.get_prediction_fn(model, pred_mode = pred_mode)



torch.no_grad()

obj = RegressionGame(X = X[0:k], y=None, function = predict_fn, transform = torch.as_tensor)

X_samp = X[k:(N+k)]

shapley_values = np.empty((0, X.shape[1]))
partial_residuals = np.empty((0, X.shape[1]))
games = np.empty((0, 2 ** X.shape[1]))

print("SHape")
print(X.shape[1])

print("  ..ok!")
print("Generating explanations..")

for i in range(0, N):
    example_row = X_samp[i,:].reshape((1,X_samp.shape[1]))
    game = obj.getKernelSHAPGame(example_row)
    games = np.append(games, game.reshape((1,game.shape[0])), axis = 0)
    results, residualGame, origGame = getShapleyProjection(game)
    shapley_values = np.append(shapley_values,
                               np.array([np.flip(results[-1])]), axis=0)
    partial_residuals = np.append(partial_residuals,
                                  np.array([np.flip(norm(residualGame, axis = 0)/norm(origGame, axis = 0))]), axis = 0)
    print("%s/%s samples done." % (i+1, N))
    

    if i % 100 == 0:
        pd.DataFrame(X_samp).to_csv('data/llm_input.csv')
        pd.DataFrame(shapley_values).to_csv('data/llm_shapley_values.csv')
        pd.DataFrame(partial_residuals).to_csv('data/llm_partial_residuals.csv')

print(" Explanations saved to data/llm_*.csv!")

pd.DataFrame(X_samp).to_csv('data/llm_input.csv')
pd.DataFrame(shapley_values).to_csv('data/llm_shapley_values.csv')
pd.DataFrame(partial_residuals).to_csv('data/llm_partial_residuals.csv')

"""
TODO: 
1. How to get base line features for the text generation process? 
2. What is the appropriate metric for shapley score. 
"""

In [ ]:
partial_residuals

In [ ]:
# predict_fn = llm_helper.get_prediction_fn(model, pred_mode = 1)



# torch.no_grad()


# # obj = RegressionGame(X = X[0:k], y=y[0:k], function = predict_fn, transform = torch.as_tensor)
# obj = RegressionGame(X = X[0:k], y=None, function = predict_fn, transform = torch.as_tensor)

# X_samp = X[k:(N+k)]

# shapley_values = np.empty((0, X.shape[1]))
# partial_residuals = np.empty((0, X.shape[1]))
# games = np.empty((0, 2 ** X.shape[1]))

# print("SHape")
# print(X.shape[1])

# print("  ..ok!")
# print("Generating explanations..")

# for i in range(0, N):
#     example_row = X_samp[i,:].reshape((1,X_samp.shape[1]))
#     game = obj.getKernelSHAPGame(example_row)
#     games = np.append(games, game.reshape((1,game.shape[0])), axis = 0)
#     results, residualGame, origGame = getShapleyProjection(game)
#     shapley_values = np.append(shapley_values,
#                                np.array([np.flip(results[-1])]), axis=0)
#     partial_residuals = np.append(partial_residuals,
#                                   np.array([np.flip(norm(residualGame, axis = 0)/norm(origGame, axis = 0))]), axis = 0)
#     print("%s/%s samples done." % (i+1, N))

# print(" Explanations saved to data/llm_*.csv!")

# pd.DataFrame(X_samp).to_csv('data/llm_input.csv')
# pd.DataFrame(shapley_values).to_csv('data/llm_shapley_values.csv')
# pd.DataFrame(partial_residuals).to_csv('data/llm_partial_residuals.csv')

# """
# TODO: 
# 1. How to get base line features for the text generation process? 
# 2. What is the appropriate metric for shapley score. 
# 3. issue here, check why I was not able to make k = 40. model is probably not able to take more than 25. Check
# """

In [ ]:
shapley_values

In [ ]:
partial_residuals

In [ ]:
                            
# s_l[k] = '[MASK]'
s_l = "What is [MASK] name?"
print(s_l)
og =  tokenizer(s_l, padding=False,  truncation=True, max_length=seq_len, return_tensors ='pt').input_ids[0]
if cuda:
    og = og.cuda()

print(og.shape)
og = og.reshape(1, -1)
mask_token_index = torch.nonzero(og[0] == tokenizer.mask_token_id, as_tuple=False).squeeze(-1)
print(mask_token_index)

print(og.shape)
logits = model(og).logits
print(outputs.shape)

# print(result)
print(logits.shape)
print(mask_token_index)
print(logits[0, mask_token_index, :].shape)
print(logits.argmax(axis=-1)[0, mask_token_index])
print('asdasd')
abc = logits[0, mask_token_index, :].argmax(axis=-1)
print(abc)
abc = logits.argmax(axis=-1)
print(abc)
print(tokenizer.decode(abc[0]))

